In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import spacy
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from textblob import TextBlob

In [2]:
en_train = "../data/processed/processed_CT24_checkworthy_english/processed_train.tsv"
du_train = "../data/processed/processed_CT24_checkworthy_dutch/processed_dutch_train.tsv"  # noqa
es_train = "../data/processed/processed_CT24_checkworthy_spanish/processed_spanish_train.tsv"  # noqa
ar_train = "../data/processed/processed_CT24_checkworthy_arabic/processed_arabic_train.tsv"  # noqa

en_test = "../data/processed/processed_CT24_checkworthy_english/processed_dev.tsv"
du_test = "../data/processed/processed_CT24_checkworthy_dutch/processed_dutch_dev.tsv"
es_test = "../data/processed/processed_CT24_checkworthy_spanish/processed_spanish_dev.tsv"  # noqa
ar_test = "../data/processed/processed_CT24_checkworthy_arabic/processed_arabic_dev.tsv"

en_dev_test = "../data/processed/processed_CT24_checkworthy_english/processed_dev_test.tsv"  # noqa
du_dev_test = "../data/processed/processed_CT24_checkworthy_dutch/processed_dutch_dev_test.tsv"  # noqa
es_dev_test = "../data/processed/processed_CT24_checkworthy_spanish/processed_spanish_dev_test.tsv"  # noqa
ar_dev_test = "../data/processed/processed_CT24_checkworthy_arabic/processed_arabic_dev_test.tsv"  # noqa

en_train = pd.read_csv(en_train, sep="\t")
du_train = pd.read_csv(du_train, sep="\t")
es_train = pd.read_csv(es_train, sep="\t")
ar_train = pd.read_csv(ar_train, sep="\t")

en_test = pd.read_csv(en_test, sep="\t")
du_test = pd.read_csv(du_test, sep="\t")
es_test = pd.read_csv(es_test, sep="\t")
ar_test = pd.read_csv(ar_test, sep="\t")

en_dev_test = pd.read_csv(en_dev_test, sep="\t")
du_dev_test = pd.read_csv(du_dev_test, sep="\t")
es_dev_test = pd.read_csv(es_dev_test, sep="\t")
ar_dev_test = pd.read_csv(ar_dev_test, sep="\t")

In [3]:
en_test

,Sentence_id,tweet_text,class_label,hashtags,mentions,text_length,text_length_category,class_label_encoded,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,26,"You know, I saw a movie - Crocodile Dundee.""",No,NaN,NaN,46,41-50,0,0,0,0
1,80,We're consuming 50 percent of the world's coca...,Yes,NaN,NaN,50,41-50,1,0,0,0
2,129,That answer was about as clear as Boston harbor.,No,NaN,NaN,48,41-50,0,0,0,0
3,131,Let me help the governor.,No,NaN,NaN,25,21-30,0,0,0,0
4,172,We've run up more debt in the last eight years...,Yes,NaN,NaN,125,100+,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1027,28958,He has promised a trillion dollars out of the ...,Yes,NaN,NaN,130,100+,1,0,0,0
1028,28965,(LAUGHTER) I -- there's an old high school deb...,No,NaN,NaN,131,100+,0,0,0,0
1029,29011,"Well, can I answer that?",No,NaN,NaN,26,21-30,0,0,0,0
1030,29060,I look forward to the final weeks of this camp...,No,NaN,NaN,51,51-60,0,0,0,0


In [4]:
[
    en_train.columns,
    du_train.columns,
    es_train.columns,
    ar_train.columns,
    en_test.columns,
    du_test.columns,
    es_test.columns,
    ar_test.columns,
    en_dev_test.columns,
    du_dev_test.columns,
    es_dev_test.columns,
    ar_dev_test.columns,
]

[Index(['Sentence_id', 'tweet_text', 'class_label', 'text_length', 'hashtags',
        'mentions', 'text_length_category', 'class_label_encoded',
        'hashtags_frequency', 'hashtags_sentiment', 'hashtags_topics'],
       dtype='object'),
 Index(['tweet_id', 'tweet_text', 'class_label', 'text_length', 'hashtags',
        'mentions', 'text_length_category', 'class_label_encoded',
        'hashtags_frequency', 'hashtags_sentiment', 'hashtags_topics'],
       dtype='object'),
 Index(['tweet_id', 'tweet_text', 'class_label', 'text_length', 'hashtags',
        'mentions', 'text_length_category', 'class_label_encoded',
        'hashtags_frequency', 'hashtags_sentiment', 'hashtags_topics'],
       dtype='object'),
 Index(['tweet_id', 'tweet_text', 'class_label', 'text_length', 'hashtags',
        'mentions', 'text_length_category', 'class_label_encoded',
        'hashtags_frequency', 'hashtags_sentiment', 'hashtags_topics'],
       dtype='object'),
 Index(['Sentence_id', 'tweet_text', 'cla

In [5]:
trains = [en_train, du_train, es_train, ar_train]
tests = [en_test, du_test, es_test, ar_test]
dev_tests = [en_dev_test, du_dev_test, es_dev_test, ar_dev_test]


cols_to_keep = [
    "tweet_text",
    "Text",
    "class_label",
    "hashtags",
    "mentions",
    "text_length",
    "text_length_category",
    "hashtags_frequency",
    "hashtags_sentiment",
    "hashtags_topics",
]

for train in [
    en_train,
    du_train,
    es_train,
    ar_train,
    en_test,
    du_test,
    es_test,
    ar_test,
    en_dev_test,
    du_dev_test,
    es_dev_test,
    ar_dev_test,
]:
    cols_to_drop = [col for col in train.columns if col not in cols_to_keep]
    train.drop(columns=cols_to_drop, inplace=True)
    train.rename(
        columns={"Text": "text", "class_label": "label", "tweet_text": "text"},
        inplace=True,
    )

merged_train = pd.concat(trains).sample(frac=1).reset_index(drop=True)
merged_test = pd.concat(tests).sample(frac=1).reset_index(drop=True)
merged_dev_test = pd.concat(dev_tests).sample(frac=1).reset_index(drop=True)

merged_train["label"] = merged_train["label"].apply(lambda x: 1 if x == "Yes" else 0)
merged_test["label"] = merged_test["label"].apply(lambda x: 1 if x == "Yes" else 0)
merged_dev_test["label"] = merged_dev_test["label"].apply(
    lambda x: 1 if x == "Yes" else 0
)

merged_train.to_csv("../data/processed/merged_train.tsv", sep="\t", index=False)
merged_test.to_csv("../data/processed/merged_test.tsv", sep="\t", index=False)
merged_dev_test.to_csv("../data/processed/merged_dev_test.tsv", sep="\t", index=False)